### Parkinson's Mouse Tutorial - Import & Demux

Run this notebook in `qiime2-2022.11`.

Well be working through the [pd-mouse tutorial](https://docs.qiime2.org/2022.11/tutorials/pd-mice/).

*Note: did you run `jupyter serverextension enable --py qiime2 --sys-prefix` before getting here?*

Also, see the [Jupyter Markdown documentation](https://jupyter.brynmawr.edu/services/public/dblank/Jupyter%20Notebook%20Users%20Manual.ipynb).

In [1]:
from os import getcwd, listdir, chdir, mkdir
import qiime2 as q2

In [2]:
getcwd()

'/home/fmshaik/BMIG-62033'

In [3]:
listdir()

['01-Parkinson-Mouse-Tutorial-Import-Demux.ipynb',
 'processed',
 '04-Parkinson-Mouse-Tutorial-Diff-Abund.ipynb',
 '.ipynb_checkpoints',
 '02-Parkinson-Mouse-Tutorial-Taxonomy-Phylogeny.ipynb',
 '03-Parkinson-Mouse-Tutorial-Diversity.ipynb']

In [4]:
mkdir('./processed')

FileExistsError: [Errno 17] File exists: './processed'

In [ ]:
chdir('./processed')
getcwd()

## Download and View Metadata

We'll use `wget` to download the metadata file, and then visualize it in onw of two ways:
 - [QIIME 2 View Website](https://view.qiime2.org/)
 - [QIIME 2 CLI / Utilities](https://docs.qiime2.org/2022.11/tutorials/utilities/)
 - [QIIME 2 API](https://docs.qiime2.org/2022.11/interfaces/artifact-api/)
 
 *Note: If you are running this notebook on the HPC, you may need to copy and paste these commands into the "Grace Shell Access" under the "Clusters" menu of the Grace HPC Portal page. Make sure you are downloading the files into the appropriate directory. Aalternatively, simply download the files to you computer and use Jupyter Lab to upload the files.*

In [ ]:
# Download Metadata
! wget \
    -O "metadata.tsv" \
    "https://data.qiime2.org/2022.11/tutorials/pd-mice/sample_metadata.tsv"

In [ ]:
# Peek at the metadata
! qiime tools inspect-metadata metadata.tsv

**Make metadata Visualization**

In [ ]:
! qiime metadata tabulate \
  --m-input-file metadata.tsv \
  --o-visualization metadata.qzv

In [ ]:
! qiime tools peek metadata.qzv

In [ ]:
# Visualize via API
q2.Visualization.load('metadata.qzv')

## Import data into QIIME 2

We will import:
 - [Manifest File](https://docs.qiime2.org/2022.11/tutorials/importing/#fastq-manifest-formats)
 - Demultiplexed Sequences (contrast to Multiplexed Sequences)
 
See the [Importing Data Tutorial](https://docs.qiime2.org/2022.11/tutorials/importing/#importing-data) for more information.

In [ ]:
# get manifest file
!wget \
  -O "manifest.tsv" \
  "https://data.qiime2.org/2022.11/tutorials/pd-mice/manifest"

In [ ]:
# get demultiplexed sequences
!wget \
  -O "demultiplexed_seqs.zip" \
  "https://data.qiime2.org/2022.11/tutorials/pd-mice/demultiplexed_seqs.zip"

In [ ]:
# unzip sequences
! unzip demultiplexed_seqs.zip

In [ ]:
! head manifest.tsv

**Import and Summarize Data**

In [ ]:
! qiime tools import \
  --type "SampleData[SequencesWithQuality]" \
  --input-format SingleEndFastqManifestPhred33V2 \
  --input-path ./manifest.tsv \
  --output-path ./demux_seqs.qza

In [ ]:
! qiime demux summarize \
  --i-data ./demux_seqs.qza \
  --o-visualization ./demux_seqs.qzv

## Denoising Sequence data

 - DADA2 approach as outlined in the tutorial.
 - Alternate trimming w/ DADA2.
 - Using deblur w/ default trimming.

### Default

In [ ]:
getcwd()

In [ ]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs ./demux_seqs.qza \
    --p-trunc-len 150 \
    --p-n-threads 4 \
    --o-table ./dada2_table.qza \
    --o-representative-sequences ./dada2_rep_set.qza \
    --o-denoising-stats ./dada2_stats.qza \
    --verbose

In [ ]:
# summarize denoising stats
! qiime metadata tabulate \
    --m-input-file ./dada2_stats.qza  \
    --o-visualization ./dada2_stats.qzv

In [ ]:
# summarize ESV table
! qiime feature-table summarize \
    --i-table ./dada2_table.qza \
    --m-sample-metadata-file ./metadata.tsv \
    --o-visualization ./dada2_table.qzv

In [ ]:
! qiime tools export \
    --input-path ./dada2_table.qza \
    --output-path ./dada2_table_export

In [ ]:
! qiime feature-table tabulate-seqs \
    --i-data ./dada2_rep_set.qza \
    --o-visualization ./dada2_rep_set.qzv

In [ ]:
! biom convert --to-tsv \
    -i ./dada2_table_export/feature-table.biom \
    -o ./dada2_table_export/feature-table.txt

In [ ]:
! qiime feature-table transpose \
    --i-table ./dada2_table.qza \
    --o-transposed-feature-table ./dada2_table_transposed.qza

! qiime metadata tabulate \
    --m-input-file ./dada2_table_transposed.qza  \
    --o-visualization ./dada2_table_transposed_tab.qzv

! qiime tools export \
    --input-path ./dada2_table_transposed.qza \
    --output-path ./dada2_table_transposed_export

! biom convert --to-tsv \
    -i ./dada2_table_transposed_export/feature-table.biom \
    -o ./dada2_table_transposed_export/feature-table.txt

In [ ]:
! qiime metadata tabulate \
    --m-input-file ./dada2_table.qza  \
    --o-visualization ./dada2_table_tab.qzv

### Alternate Trimming w/ DADA2

In [ ]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs ./demux_seqs.qza \
    --p-trim-left 30 \
    --p-trunc-len 130 \
    --o-table ./dada2_table_alt.qza \
    --o-representative-sequences ./dada2_rep_set_alt.qza \
    --o-denoising-stats ./dada2_stats_alt.qza \
    --verbose

In [ ]:
# summarize denoising stats
! qiime metadata tabulate \
    --m-input-file ./dada2_stats_alt.qza  \
    --o-visualization ./dada2_stats_alt.qzv

In [ ]:
q2.Visualization.load('dada2_stats_alt.qzv')

In [ ]:
# summarize ESV table
! qiime feature-table summarize \
    --i-table ./dada2_table_alt.qza \
    --m-sample-metadata-file ./metadata.tsv \
    --o-visualization ./dada2_table_alt.qzv

In [ ]:
q2.Visualization.load('dada2_table_alt.qzv')

### deblur w/ default

In [ ]:
! qiime quality-filter q-score \
    --i-demux ./demux_seqs.qza \
    --o-filtered-sequences demux-seqs-deblur.qza \
    --o-filter-stats demux-deblur-stats.qza

In [ ]:
# Defaults to Greengenes. 
#    If you want to use SILVA or another ref db, then use:
#    `qiime deblur denoise-other`
#    silva files are located here: https://docs.qiime2.org/2022.11/data-resources/
! qiime deblur denoise-16S \
    --i-demultiplexed-seqs demux-seqs-deblur.qza \
    --p-trim-length 131 \
    --o-representative-sequences rep-seqs-deblur.qza \
    --o-table table-deblur.qza \
    --p-sample-stats \
    --o-stats deblur-stats.qza

In [ ]:
! qiime metadata tabulate \
    --m-input-file demux-deblur-stats.qza \
    --o-visualization demux-deblur-stats.qzv

! qiime deblur visualize-stats \
    --i-deblur-stats deblur-stats.qza \
    --o-visualization deblur-stats.qzv

In [ ]:
q2.Visualization.load('demux-deblur-stats.qzv')

In [ ]:
q2.Visualization.load('deblur-stats.qzv')

In [ ]:
! qiime feature-table summarize \
    --i-table table-deblur.qza \
    --o-visualization table-deblur.qzv \
    --m-sample-metadata-file metadata.tsv

! qiime feature-table tabulate-seqs \
    --i-data rep-seqs-deblur.qza \
    --o-visualization rep-seqs-deblur.qzv

In [ ]:
q2.Visualization.load('table-deblur.qzv')

In [ ]:
q2.Visualization.load('rep-seqs-deblur.qzv')